In [1]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 2, 26)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name spd reason qty target current percent active'.split()

In [3]:
format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}'
}

In [4]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

,id,trade,name,qty,price,active,reason,market
0,25,S,WHART,"10,000",12.80,1,DOS,SET


In [5]:
names = orders['name']
name = names.to_string(index=False)

sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,10.68,I,0.00,12.80,19.32,0.37,12.50,"10,000",-5,4,"10,000",0,0,DOS,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [6]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

25

In [7]:
sql = '''
INSERT INTO orders
SELECT id, 'B', name, qty, buy_target, 0, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
rp = conlite.execute(sql)
rp.rowcount

10

In [8]:
sql = '''
INSERT INTO orders
SELECT id,'S',name, qty, sell_target, 0, reason, market
FROM stocks
WHERE status IN ('I','S')'''
rp = conlite.execute(sql)
rp.rowcount

12

In [9]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_orders.shape[0]

22

In [10]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### After call Ord-log.bat

In [11]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [12]:
df = pd.read_csv('../data/orders-log.csv')
#df.rename(columns = {'col1': 'name', 'col2': 'spd','col3': 'reason', 'col4': 'market',
#                     'col5': 'qty','col6': 'target','col7': 'current', 'col8': 'chg',
#                     'col9': 'percent', 'col10': 'active'}, inplace = True)
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [13]:
df[colt]

,trans,name,spd,reason,qty,target,current,percent,active
0,Buy,ASP,-3,6L,30000,3.70,3.76,+2.17%,0
1,Buy,BAM,-5,NaN,10000,20.50,21.00,+1.94%,0
2,Buy,GLOBAL,-5,NaN,9000,19.40,19.90,-5.69%,0
3,Buy,IMH,-5,NaN,9000,21.00,21.50,+11.98%,0
4,Buy,MAKRO,-11,RD05pct,1500,39.75,42.50,+1.19%,0
5,Buy,MEGA,-11,10pct,1500,40.50,43.25,+1.17%,0
6,Buy,RCL,-13,ROUND,1500,44.50,47.75,+8.52%,0
7,Buy,SENA,-5,NaN,30000,4.90,5.00,+2.88%,0
8,Buy,TMT,-4,ROUND,15000,10.50,10.90,+2.83%,0
9,Buy,TPIPL,-5,NaN,60000,1.70,1.75,+1.16%,0
